Load a random chunks file and plot the position of each chunk within each file.

In [1]:
import os, json
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

videoPath = '/Users/cudmore/Dropbox/PiE/scrambled'
chunkFile = os.path.join(videoPath, 'randomChunks.txt')

with open(chunkFile) as f:
    chunkDict = json.load(f)

chunkOrder = chunkDict['chunkOrder'] # random order the chunks were scored in
#print('total number of chunks', len(chunkOrder))

fileDict = {} # one key per file, each key holds 'x' list and 'y' list
yIndex = 0 # keep track of number of files we add to fileDict (use for y-axis)
for chunk in chunkDict['chunks']:
        path = chunk['path']
        index = chunk['index']
        fileName = os.path.basename(path)
        fileName = fileName.replace('.mp4', '')
        colorIdx = chunkOrder.index(index) # find the index into random list of chunks
        if fileName in fileDict:
            # append chunk to list
            fileDict[fileName]['numChunks'] += 1 # keep track of number of chunks we encounter
            fileDict[fileName]['x'].append(chunk['startFrame'])
            fileDict[fileName]['x'].append(chunk['stopFrame'])
            fileDict[fileName]['x'].append(np.NaN)
            fileDict[fileName]['y'].append(yIndex)
            fileDict[fileName]['y'].append(yIndex)
            fileDict[fileName]['y'].append(yIndex)
            fileDict[fileName]['colors'].append(colorIdx)
            fileDict[fileName]['colors'].append(colorIdx)
            fileDict[fileName]['colors'].append(colorIdx)
        else:
            # initialize new list from chunk
            yIndex += 1
            fileDict[fileName] = {}
            fileDict[fileName]['numChunks'] = 1 # keep track of number of chunks we encounter
            fileDict[fileName]['x'] = [chunk['startFrame'], chunk['stopFrame'], np.NaN]
            fileDict[fileName]['y'] = [yIndex, yIndex, yIndex]
            fileDict[fileName]['colors'] = [colorIdx, colorIdx, colorIdx] 

In [2]:
def myPlot(myPlotMode):
    #myPlotMode in ['lines', 'lines+markers']

    # make a different trace for each file
    traces = [] # each file is in a different trace
    for idx, file in enumerate(fileDict):    
        if idx == 0:
            myColorBar = dict(title='Chunk Order')
        else:
            myColorBar = {}
        newTrace = go.Scatter(
            x = fileDict[file]['x'],
            y = fileDict[file]['y'],
            mode = myPlotMode,
            name = file + '(n=' + str(fileDict[file]['numChunks']) + ')',
            marker=dict(
                size=10,
                cmax=len(chunkOrder),
                cmin=0,
                color = fileDict[file]['colors'],
                colorbar = myColorBar,
                colorscale='Jet'
            ),
            line = dict(
                width = 10,
            )
        )
        traces.append(newTrace)

    data = traces
    layout = go.Layout(showlegend=False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, filename='chunks-per-file')


In [3]:
myPlot('lines')
myPlot('lines+markers')